In [15]:
from tqdm import tqdm
import math
import network
from network.deeplabv3 import DeepLabv3Plus
import utils
import os
import random
import argparse
import numpy as np

from torch.utils import data
#from datasets import VOCSegmentation, Cityscapes
#from utils import ext_transforms as et
#from metrics import StreamSegMetrics

import torch
import torch.nn as nn
import torch.functional as F
#from utils.visualizer import Visualizer

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import glob
import yaml
from datetime import datetime, timezone, timedelta
import yaml

def load_yaml(path):
    with open(path, 'r', encoding='utf-8') as f:
        return yaml.load(f, Loader=yaml.FullLoader)

c:\Users\Admin\anaconda3\envs\imgseg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## hide

In [6]:
def get_harbor_idx(data_path, train = True, is_label = False, label_num = 5):
    if train:
        if is_label:
            classes = ['ship', 'container_truck', 'forklift', 'reach_stacker']
            image_path = glob(os.path.join(root, 'train', 'labeled_images', '*.jpg'))
            image_idx_list = list(map(lambda x : x.split('/')[-1].split('.')[0], image_path))
            train_idx = []
            valid_idx = []
            for c in classes:
                matched_idx = [i for i in image_idx_list if c in i]
                train_idx.extend(matched_idx[label_num:])
                valid_idx.extend(matched_idx[:label_num])
            return train_idx, valid_idx
        else:
            image_path = glob(os.path.join(root, 'train', 'unlabeled_images', '*.jpg'))
            train_idx = list(map(lambda x: x.split('/')[-1].split('.')[0], image_path))
            return train_idx
    else:
        image_path = glob(os.path.join(root, 'test', 'images', '*.jpg'))
        test_idx = list(map(lambda x: x.split('/')[-1].split('.')[0], image_path))
        return test_idx



class BuildDataLoader:
    def __init__(self, num_labels, dataset_path, batch_size):
        self.data_path = dataset_path
        self.im_size = [513, 513]
        self.crop_size = [430, 430]
        self.num_segments = 5
        self.scale_size = (0.5, 1.5)
        self.batch_size = batch_size
        self.train_l_idx, self.valid_l_idx = get_harbor_idx(self.data_path, train=True, is_label=True, label_num=num_labels)
        self.train_u_idx = get_harbor_idx(self.data_path, train=True, is_label=False)
        self.test_idx = get_harbor_idx(self.data_path, train=False)

        if num_labels == 0:  # using all data
            self.train_l_idx = self.train_u_idx

    def build(self, supervised=False):
        train_l_dataset = BuildDataset(self.data_path, self.train_l_idx,
                                       crop_size=self.crop_size, scale_size=self.scale_size,
                                       augmentation=True, train=True, is_label=True)
        train_u_dataset = BuildDataset(self.data_path, self.train_u_idx,
                                       crop_size=self.crop_size, scale_size=(1.0, 1.0),
                                       augmentation=False, train=True, is_label=False)
        valid_l_dataset = BuildDataset(self.data_path, self.valid_l_idx,
                                       crop_size=self.crop_size, scale_size=self.scale_size,
                                       augmentation=False, train=True, is_label=True)
        test_dataset    = BuildDataset(self.data_path, self.test_idx,
                                       crop_size=self.im_size, scale_size=(1.0, 1.0),
                                       augmentation=False, train=False, is_label=True)

        if supervised:  # no unlabelled dataset needed, double batch-size to match the same number of training samples
            self.batch_size = self.batch_size * 2

        num_samples = self.batch_size * 200  # for total 40k iterations with 200 epochs
        # num_samples = self.batch_size * 2
        train_l_loader = torch.utils.data.DataLoader(
            train_l_dataset,
            batch_size=self.batch_size,
            sampler=sampler.RandomSampler(data_source=train_l_dataset,
                                          replacement=True,
                                          num_samples=num_samples),
            drop_last=True,)


        valid_l_loader = torch.utils.data.DataLoader(
            valid_l_dataset,
            batch_size=self.batch_size,
            sampler=sampler.RandomSampler(data_source=valid_l_dataset,
                                          replacement=True,
                                          num_samples=num_samples),
            drop_last=True,)

        if not supervised:
            train_u_loader = torch.utils.data.DataLoader(
                train_u_dataset,
                batch_size=self.batch_size,
                sampler=sampler.RandomSampler(data_source=train_u_dataset,
                                              replacement=True,
                                              num_samples=num_samples),
                drop_last=True,)

        test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
        )
        if supervised:
            return train_l_loader, valid_l_loader, test_loader
        else:
            return train_l_loader, train_u_loader, valid_l_loader, test_loader

In [ ]:
import torch.utils.data as data
from collections import namedtuple

class Comp_dataset(data.Dataset):
    DatasetClass = namedtuple('DataClass', ['name', 'id'])
    classes = [
        DatasetClass('unlabeled', 0),
        DatasetClass('container_truck', 1),
        DatasetClass('forklift', 2),
        DatasetClass('reach_stacker', 3),
        DatasetClass('ship', 4)
    ]

    id_to_train_id = np.array([c.train_id for c in classes])

    def __init__(self, root, transform = None):
        # Base directory
        self.root = os.path.expanduser(root)
        self.image_dir = os.path.join(self.root, 'data')
        self.num_classes = 5
        self.transform = transform

In [ ]:
def get_dataset():
    train_transform = extcompose([])

    train_dataset = Comp_dataset(root=, split=, transform=)
    return train_dataset, val_dataset

In [ ]:
def save_yaml(path, obj):
	with open(path, 'w') as f:
		yaml.dump(obj, f, sort_keys=False)
		

def load_yaml(path):
	with open(path, 'r', encoding='utf-8') as f:
		return yaml.load(f, Loader=yaml.FullLoader)

## Model


In [25]:
# from modeling.backbone import resnet
model_urls = {'resnet101': "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth"}

# Conv parameters = Kernel size x Kernel Size x Input Channel x Output Channel
# BottleNeck is 1x1 Conv: 1 x 1 x Input Channel x Output Channel
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, stride = 1, dilation=1, downsample=None, BatchNorm=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = BatchNorm(planes)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=stride, dilation=dilation,
            padding=dilation, bias=False,
        )
        self.bn2 = BatchNorm(planes)
        self.conv3 = nn.Conv2d(planes,planes*4, kernel_size=1, bias=False)
        self.bn3 = BatchNorm(planes*4)
        self.relu = nn.ReLU(inplace= True)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual # f(x) + x
        out = self.relu(out)

        return out

# Resnet class. make layers
class ResNet(nn.Module):
    def __init__(self, block, layers, output_stride, BatchNorm, pretrained = True, url = None):
        self.inplanes = 64
        super(ResNet, self).__init__()
        blocks = [1, 2, 4]
        if output_stride == 16:
            strides = [1, 2, 2, 1]
            dilations = [1, 1, 1, 2]
        elif output_stride == 8:
            strides = [1, 2, 1, 1]
            dilations = [1, 1, 2, 4]
        else:
            raise NotImplementedError
    
        # Modules
        self.conv1 = nn.Conv2d(3, 64, stride=2, padding=3, bias=False)
        self.bn1 = BatchNorm(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], stride = strides[0],
                                        dilation = dilations[0], BatchNorm=BatchNorm,
        )
        self.layer2 = self._make_layer(block, 128, layers[1], stride = strides[1],
                                        dilation= dilations[1], BatchNorm=BatchNorm)
        self.layer3 = self._make_layer(block, 128, layers[2], stride = strides[2],
                                        dilation= dilations[2], BatchNorm=BatchNorm)
        self.layer4 = self._make_layer(block, 128, layers[2], stride = strides[2],
                                        dilation= dilations[2], BatchNorm=BatchNorm)

        
        self._init_weight()
        if pretrained:
            print("load pretrained RESNET model's weight")
            self._load_pretrained_model(url)

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1, BatchNorm=None):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            #self.inplanes: input_channel
            downsample = nn.Sequential(                
                nn.Conv2d(
                    self.inplanes, planes*block.expension,
                    kernel_size=1, stride=stride,bias=False,),
                BatchNorm(planes*block.expansion),
            )
            

        layers = []
        layers.append(
            block(self.inplanes, planes, stride, dilation, downsample, BatchNorm=BatchNorm)
        )
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(
                block(self.inplanes, planes, dilation=dilation, BatchNorm=BatchNorm)
            )

        return nn.Sequential(*layers)

    def _make_MG_unit(
        self, block, planes, blocks, stride=1, dilation=1, BatchNorm=None
    ):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.inplanes, planes*block.expansion, kernel_size=1,
                    stride = stride, bias=False,
                ),
                BatchNorm(planes * block.expansion),
            )
        
        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride,dilation=blocks[0]*dilation,
                downsample=downsample,BatchNorm = BatchNorm,
            )
        )
        self.inplanes =planes *block.expansion
        for i in range(1, len(blocks)):
            layers.append(
                block(self.inplanes,planes,stride = 1,
                      dilation=blocks[i]*dilation, BatchNorm=BatchNorm,)
            )
        return nn.Sequential(*layers)


    @torch.cuda.amp.autocast()
    def forward(self, input, intercept=False):
        x = self.conv1(input) # Conv2d(3, 64)
        x = self.bn1(x) # Batchnorm(64)
        x = self.relu(x) # ReLU
        x = self.maxpool(x) # MaxPool2d

        x_1 = self.layer1(x)
        x_2 = self.layer2(x_1)
        x_3 = self.layer3(x_2)
        x_4 = self.layer4(x_3)
        if intercept is True:
            return x_1, x_3, x_4
        return x_4, x_1

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2.0 / n))
            elif isinstance(m, SynchronizedBatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

def ResNet101(output_stride, BatchNorm, pretrained=True):
    r'''Construct a ResNet-101 model
    Args:
        pretrained(bool): If True, returns a model pre-trained on ImageNet
    '''
    print("Using Resnet101 as a Backbone Network")
    model = ResNet(
        Bottleneck, [3, 4, 23, 3], output_stride, BatchNorm,
        pretrained=pretrained, url=model_urls['resnet101'],
    )
    return model

In [22]:
from models.sync_batchnorm.batchnorm import SynchronizedBatchNorm2d
from models.assp import build_aspp
from models.decoder import build_decoder

class classifier(nn.Module):
    def __init__(self, num_classes = 5, out_dim = 256):
        super(classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Conv2d(out_dim, out_dim, kernel_size=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_dim, num_classes, kernel_size=1, stride=1),
        )

    @torch.cuda.amp.autocast()
    def forward(self, x, expand = None):
        cls = self.classifier(x)
        return cls # b x num_class

class Projector(nn.Module):
    def __init__(self, out_dim = 256):
        super(Projector, self).__init__()

        self.projector = nn.Sequential(
            nn.Conv2d(out_dim, out_dim, kernel_size=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_dim, 128, kernel_size=1, stride=1),
        )
    
    @torch.cuda.amp.autocast()
    def forward(self, x):
        x = self.projector(x)
        return x


class DeepLab_temp(nn.Module):
    def __init__(self, output_stride = 16):
        super(DeepLab_temp, self).__init__()
        BatchNorm = SynchronizedBatchNorm2d

        self.backbone = ResNet101(output_stride, BatchNorm)
    
    @torch.cuda.amp.autocast()
    def forward(self, input):
        x, low_level_feat = self.backbone(input)
        return x, low_level_feat

    def get_backbone_params(self):
        modules = [self.backbone]
        for i in range(len(modules)):
            #nn.Module.named_modules: returns an iterator over all modules in the network
            for m in modules[i].named_modules():
                if(isinstance(m[1], nn.Conv2d)
                   or isinstance(m[1], nn.BatchNorm2d)
                   or isinstance(m[1], nn.BatchNorm2d)
                ):
                    for p in m[1].parameters():
                        if p.requires_grad:
                            yield p

class Decoder(nn.Module):
    def __init__(
        self, backbone="resnet101", output_stride=16, num_classes=256,num_cls = 5
    ):
        BatchNorm = SynchronizedBatchNorm2d

        super(Decoder, self).__init__()
        self.aspp = build_aspp(backbone, output_stride, BatchNorm)
        self.decoder = build_decoder(num_classes, backbone, BatchNorm)
        self.cls = classifier(num_classes=num_cls)
        self.proj = Projector()

    @torch.cuda.amp.autocast()
    def forward(self, x, low_level_feat, gt_mode = False):
        x = self.aspp(x, gt_mode)
        x_ = self.decoder(x, low_level_feat, gt_mode)
        cls = self.cls(x_)
        proj = self.proj(x_)
        return cls, proj
    
    def get_other_params(self):
        modules = [self.aspp, self.decoder, self.cls, self.proj]
        for i in range(len(modules)):
            for m in modules[i].named_modules():
                if(isinstance(m[1], nn.Conv2d)
                   or isinstance(m[1], nn.BatchNorm2d)
                   or isinstance(m[1], nn.BatchNorm2d)
                ):
                    for p in m[1].parameters():
                        if p.requires_grad:
                            yield p

c:\Users\Admin\anaconda3\envs\imgseg\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [18]:
# Root directory
PROJECT_DIR = os.path.join('C:/Users/Admin/OneDrive/C Documents/GitHub/ImageClassification/deeplabv3p_torch')

# Load config
config_path = os.path.join(PROJECT_DIR, 'config', 'train_config.yml')
config = load_yaml(config_path)

# # Train Serial
# kst = timezone(timedelta(hours=9))
# train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")

# # Recorder directory
# RECORDER_DIR = os.path.join(PROJECT_DIR, 'results', 'train', train_serial)
# os.makedirs(RECORDER_DIR, exist_ok=True)

# Data directory
DATA_DIR = os.path.join("C:/Users/Admin/OneDrive/C Documents/GitHub/aivill_seg/dataset")

# GPU
#os.environ['CUDA_VISIBLE_DEVICES'] = config['gpu']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: {}".format(device))

# Setup random seed
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

Device: cpu


In [19]:
print(DATA_DIR)

C:/Users/Admin/OneDrive/C Documents/GitHub/aivill_seg/dataset


In [26]:
num_classes = 5
####
#set up model (all models are 'constructed at network.modeling)
encode = DeepLab_temp()
decode = Decoder('5').cuda()
#elnetwork = ELNetwork(num_classes).cuda()
trainable_params = [
    ## main model's parameters
    {'params': list(filter(lambda p:p.requires_grad, encode.get_backbone_params())), 'lr':0.0005},
    {'params': list(filter(lambda p:p.requires_grad, dec.get_backbone_params())), 'lr':0.005},
]

Using Resnet101 as a Backbone Network


TypeError: __init__() missing 1 required positional argument: 'kernel_size'

In [9]:


model = DeepLabv3Plus()



#####
#set up metrics
metrics = 
    
#####
#set up optimizer 
optimizer = torch.optim.AdamW(
    params=model.parameters(), lr=config['lr'], weight_decay=config['weight_decay']
)
if config['lr_policy'] == 'poly':
    scheduler = PolyLR
elif config['lr_policy'] == 'step':
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=0.1)

# Gradient scaling 그래디언트가 samll magnitudes에 
scaler = torch.cuda.amp.GradScaler()

######
#set up criterion
# criterion = utils.get_loss(config['loss_type'])
if config['loss_type'] == 'focal_loss':
    criterion = utils.FocalLoss(ignore_index = 255, size_average = True)
elif config['loss_type'] == 'cross_entropy':
    criterion = nn.CrossEntropyLoss(ignore_index=255, reduction = 'mean')

def save_ckpt(path):
    """save current model"""
    torch.save()
    print("Model saved as {}".format(path))

mkdir('checkpoints')

# Restore
best_score == 0.0
cur_itrs = 0
cur_epochs = 0
if config['ckpt'] not None and os.path.isfile(config['ckpt']): # if checkpoint is available
    checkpoint = torch.load(config['ckpt'], map_location = torch.device['cpu'])
    model.load_state_dict(checkpoint['model_state'])
    model = nn.DataParallel(model)
    model.to(device)
else:
    print("[!] Retrain")
    model = nn.DataParallel(model)
    model.to(device)


# train Loop



SyntaxError: invalid syntax (477171462.py, line 44)

In [ ]:
class